In [1]:
# code for autoreload of modules
%load_ext autoreload
%autoreload 2

In [2]:
import os
# Set your custom working DIR:
DIR = '/Users/jannispoltier/Documents/Studium/3. Semester/tabpfn_credit_codebase'
# DIR = r'C:\Users\u0148775\PycharmProjects\CreditScoring'
os.chdir(DIR)



In [3]:

from src.utils import set_random_seed
set_random_seed(0)

In [4]:
from src.utils import load_config
from pathlib import Path

# read dataconfig from CONFIG_DATA.yaml file:

# dataconfig = load_config('config\CONFIG_DATA.yaml')
# experimentconfig = load_config('config\CONFIG_EXPERIMENT.yaml')
# methodconfig = load_config('config\CONFIG_METHOD.yaml')
# evaluationconfig = load_config('config\CONFIG_EVALUATION.yaml')
dataconfig = load_config(Path('config') / 'CONFIG_DATA.yaml')
experimentconfig = load_config(Path('config') / 'CONFIG_EXPERIMENT.yaml')
methodconfig = load_config(Path('config') / 'CONFIG_METHOD.yaml')
evaluationconfig = load_config(Path('config') / 'CONFIG_EVALUATION.yaml')
tuningconfig = load_config(Path('config') / 'CONFIG_TUNING.yaml')


In [11]:
from src.classes.deprecated.experiment_deprecated import Experiment

# Initialize experiment object
experiment = Experiment(dataconfig=dataconfig,
                        experimentconfig=experimentconfig,
                        methodconfig=methodconfig,
                        evaluationconfig=evaluationconfig)



Experiment created at:  2025-05-26-20-23
Task: pd
Dataset: 01_gmsc
CV splits:  3


In [7]:
from src.classes.experiment import Experiment

# Initialize experiment object
experiment = Experiment(dataconfig=dataconfig,
                        experimentconfig=experimentconfig,
                        methodconfig=methodconfig,
                        evaluationconfig=evaluationconfig,
                        tuningconfig=tuningconfig)


Experiment created at:  2025-06-02-23-12
Task: pd
Dataset: 01_gmsc
CV splits:  3


In [8]:
experiment.run()

01_gmsc loaded
- Dataset has 150000 rows. Subsampling to 30000 rows.
01_gmsc preprocessed
x shape:  (30000, 10)
y shape:  (30000,)


Cross-validation loop:: 100%|██████████| 3/3 [00:00<00:00, 17.44it/s]

- Omitting rows with missing values: 6042 rows left out
---- Tuning dt ----
*Selected hyperparameters (dt)* {'criterion': 'gini', 'max_depth': 10}
---- Tuning lr ----
*Selected hyperparameters (lr)* {'penalty': 'l2', 'C': 0.1}
- Omitting rows with missing values: 6042 rows left out
- Omitting rows with missing values: 6042 rows left out


In [9]:
experiment.results

{'dt': {0: {'accuracy': 0.92622,
   'brier': 0.06837,
   'f1': 0.3046,
   'precision': 0.39938,
   'recall': 0.24618,
   'h_measure': 0.23231,
   'aucroc': 0.65479,
   'aucpr': 0.21359},
  1: {'accuracy': 0.92191,
   'brier': 0.07259,
   'f1': 0.27945,
   'precision': 0.42308,
   'recall': 0.20862,
   'h_measure': 0.24365,
   'aucroc': 0.66588,
   'aucpr': 0.23008},
  2: {'accuracy': 0.92598,
   'brier': 0.06779,
   'f1': 0.29893,
   'precision': 0.38066,
   'recall': 0.24609,
   'h_measure': 0.22213,
   'aucroc': 0.65239,
   'aucpr': 0.20387}},
 'lr': {0: {'accuracy': 0.93436,
   'brier': 0.06053,
   'f1': 0.0,
   'precision': 0.0,
   'recall': 0.0,
   'h_measure': 0.05902,
   'aucroc': 0.62487,
   'aucpr': 0.10811},
  1: {'accuracy': 0.92742,
   'brier': 0.06654,
   'f1': 0.0,
   'precision': 0.0,
   'recall': 0.0,
   'h_measure': 0.05911,
   'aucroc': 0.63109,
   'aucpr': 0.1199},
  2: {'accuracy': 0.93587,
   'brier': 0.05915,
   'f1': 0.0,
   'precision': 0.0,
   'recall': 0.0,
  

In [14]:

# Todo: implement method that writes experiment.results to a csv file / connects with WandB / ...

# Quick and dirty examplary code:

import pandas as pd

# Convert dictionary of dictionaries to a list of rows
rows = []
for model_name, splits_dict in experiment.results.items():
    for split_idx, metrics_dict in splits_dict.items():
        # Create a row dict with model and split identifiers
        row = {
            'model': model_name,
            'split': split_idx,
        }
        # Update with all metrics in that split
        row.update(metrics_dict)

        rows.append(row)

# Convert the list of row dictionaries into a DataFrame
df = pd.DataFrame(rows)

df

# Summarise df by model (along split) and get the mean and sd of all the metrics, discarding the split and sorting by mean of aucroc
df_summary = df.groupby('model').agg(['mean', 'std']).reset_index()
# df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('split')]
df_summary




model split       accuracy               brier                  f1  \
           mean  std      mean       std      mean       std      mean   
0     ann   1.0  1.0  0.932550  0.004506  0.062403  0.003812  0.000000   
1      dt   1.0  1.0  0.924703  0.002422  0.069583  0.002620  0.294327   
2     knn   1.0  1.0  0.926080  0.003625  0.073033  0.004091  0.080363   
3     lda   1.0  1.0  0.931507  0.003544  0.059013  0.002887  0.126527   
4      lr   1.0  1.0  0.932550  0.004506  0.062073  0.003929  0.000000   
5      rf   1.0  1.0  0.934263  0.003703  0.052610  0.003113  0.262270   
6     svm   1.0  1.0  0.932843  0.004464  0.061293  0.001939  0.021743   
7  tabpfn   1.0  1.0  0.936470  0.003940  0.050707  0.003045  0.293770   
8     xgb   1.0  1.0  0.932550  0.004506  0.058713  0.003197  0.000000   

            precision              recall           h_measure            \
        std      mean       std      mean       std      mean       std   
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.054427  0.010668   
1  0.013192  0.401040  0.021259  0.233630  0.021659  0.232697  0.010765   
2  0.019890  0.247737  0.015965  0.048247  0.013428  0.042327  0.011653   
3  0.025267  0.451177  0.044127  0.073877  0.016343  0.198710  0.018246   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.066383  0.012676   
5  0.027888  0.541087  0.023853  0.174070  0.026391  0.368213  0.006976   
6  0.005562  0.658273  0.132140  0.011080  0.002904  0.356070  0.016592   
7  0.043818  0.589647  0.030372  0.197573  0.040826  0.402200  0.014832   
8  0.000000  0.000000  0.000000  0.000000  0.000000  0.377150  0.005420   

     aucroc               aucpr            
       mean       std      mean       std  
0  0.621487  0.009011  0.104617  0.006050  
1  0.657687  0.007196  0.215847  0.013250  
2  0.570407  0.012304  0.091307  0.001651  
3  0.684440  0.012835  0.221907  0.024543  
4  0.633873  0.010671  0.116983  0.007833  
5  0.827767  0.006686  0.342080  0.009941  
6  0.800450  0.007351  0.348927  0.010933  
7  0.843273  0.006407  0.386983  0.011910  
8  0.822090  0.007808  0.334273  0.005463